In [1]:
import sys
sys.path.append(r'D:/projects/singletrader/')
import singletrader


{"level": "INFO", "time": "2023-03-01 16:10:32,980", "timeMillis": 1677658232861,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 148 _init_num_threads]", "message": "Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"level": "INFO", "time": "2023-03-01 16:10:32,980", "timeMillis": 1677658232861,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 160 _init_num_threads]", "message": "NumExpr defaulting to 8 threads."}
{"level": "INFO", "time": "2023-03-01 16:10:33,298", "timeMillis": 1677658232861,"module": "singletrader-service", "method": "python-service", "addition": "[__init__.py 45 init]", "message": "开始初始化路径..."}
[1812:MainThread](2023-03-01 16:10:33,637) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
{"level": "INFO", "time": "2023-03-01 16:10:33,637", "timeMillis": 1677658232861,"module": "qlib.Initialization", "method": "python-service", "addition": "[c

auth success 


In [2]:
from singletrader.datautils.qlibapi.constructor.base import MultiFactor
import pandas as pd
import warnings
from pprint import pprint
import numpy as np
warnings.filterwarnings('ignore')

fields = []
names = []


fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap']
bars = ['close','open','high','low','avg','volume','market_cap']
names += bars


In [3]:
mf = MultiFactor(field=fields,name=names,start_date='2009-01-01',end_date='2022-12-31')
data = mf._data.swaplevel(0,1)
data.index = data.index.set_names(['date','asset'])

{"level": "INFO", "time": "2023-03-01 16:10:34,173", "timeMillis": 1677658232861,"module": "root", "method": "python-service", "addition": "[base.py 71 _data]", "message": "开始生成因子数据..."}


In [5]:
def bar_resample(data,frequency,symbol_level=1,fields=None):
    """bar重采样函数"""
    data_output = {}

    if fields is None:
        fields = data.columns.tolist()
    for _field in fields:
        if _field == 'open':
            data_output[_field] = data[_field].groupby(level=symbol_level).apply(lambda x:x.droplevel(symbol_level).resample(frequency).first())
        elif _field == 'high':
            data_output[_field] = data[_field].groupby(level=symbol_level).apply(lambda x:x.droplevel(symbol_level).resample(frequency).max())
        elif _field == 'low':
            data_output[_field] = data[_field].groupby(level=symbol_level).apply(lambda x:x.droplevel(symbol_level).resample(frequency).min())
        elif _field in ['volume','money']:
            data_output[_field] = data[_field].groupby(level=symbol_level).apply(lambda x:x.droplevel(symbol_level).resample(frequency).sum())
        else:      
            data_output[_field] = data[_field].groupby(level=symbol_level).apply(lambda x:x.droplevel(symbol_level).resample(frequency).last())
    
    data_output = pd.concat(data_output,axis=1).swaplevel(0,1)
    return data_output

In [6]:
#封装好的multi factor testing 类
class MultiFactorTesting():
    def __init__(self,bar_data, factor_data,price_type='close',add_shift=1):
        self.bar_data = bar_data
        self.factor_data = factor_data
        self.price_type = price_type
        self.add_shift = add_shift
        self.all_factors = factor_data.columns.tolist()
    
    # def compute_forward_returns(self,**kwargs):
    #     price_type = kwargs.get('price_type',self.price_type)
    #     add_shift = kwargs.get('add_shift', self.add_shift)
    #     prices = self.bar_data[price_type].unstack()
    #     forward_returns = prices.pct_change().shift(-(1+add_shift)).stack()
    #     forward_returns.name = 'next_return'
    #     return forward_returns
    
    def compute_forward_returns(self,**kwargs):
        """forward return = close/open-1"""
        price_type = kwargs.get('price_type',self.price_type)
        add_shift = kwargs.get('add_shift', self.add_shift)
        prices_close = self.bar_data['close'].unstack()
        prices_open = self.bar_data['open'].unstack()
        forward_returns = (prices_close/prices_open-1).shift(-(1+add_shift)).stack()
        forward_returns.name = 'next_return'
        return forward_returns

    def get_clean_factor(self,quantiles=5,labels=None):
        if isinstance(quantiles,int):
            quantiles = {factor:quantiles for factor in self.all_factors}
        if labels is None:
            labels = {factor: list(range(1,quantiles[factor]+1)) for factor in self.all_factors}   
        
        clean_data = {factor:self.factor_data[factor].groupby(level=0).apply(lambda x:pd.qcut(x,quantiles[factor],labels=labels[factor])) for factor in self.all_factors}
        return pd.concat(clean_data,axis=1)
    
    def get_clean_factor_return(self,quantiles=5,labels=None,**kwargs):
        return_data = self.compute_forward_returns(**kwargs)
        clean_data = self.get_clean_factor(quantiles=quantiles,labels=labels)
        merge_data = pd.concat([return_data,clean_data],axis=1)
        return merge_data
    
    def compute_forward_bm_return(self,w_mkt=None,**kwargs):
        return_data = self.compute_forward_returns(**kwargs)
        if w_mkt is None:
            market_return = return_data.groupby(level=0).mean()
            market_return.name = 'mkt'
            return market_return
        else:
            w_mkt = w_mkt.reindex(return_data.index) 
            market_return = (return_data*w_mkt).groupby(level=0).sum()
            market_return.name = 'mkt'
            return market_return

## Notes
universe: all A share

period: 2010.01~2022.12 (156 months)

$ forward\_return = \frac{close_{next\_month}}{open_{next\_month}}-1$

$ distance =  1 - close / MAX(high,252days)$

market_cap: market capitalization in circulation

bechmark for equal weighted:  equal weight of all stock's forward return

bechmark for market Cap weighted:  market Cap weight of all stock's forward return

## raw data (daily)
因为计算distance需要损失过去一年数据，所以原始数据从2009年开始

In [7]:
#原始数据示例
data.head()

,,close,open,high,low,avg,volume,market_cap
date,asset,,,,,,,
2009-01-05,000001.XSHE,379.529999,374.049988,380.700012,371.709991,375.619995,871993.0,270.385193
2009-01-06,000001.XSHE,402.589996,383.040009,407.670013,380.309998,396.329987,1625468.0,286.814392
2009-01-07,000001.XSHE,390.470001,398.679993,406.489990,390.470001,397.500000,1565677.0,278.182098
2009-01-08,000001.XSHE,375.230011,381.089996,381.480011,371.320007,377.570007,1004381.0,267.322205
2009-01-09,000001.XSHE,385.000000,375.230011,388.119995,375.230011,381.089996,1546480.0,274.283691


In [7]:
#收益月偏度计算
skew = data.groupby('asset').apply(lambda x:x['close'].droplevel('asset').pct_change().resample('M').apply(lambda x:x.skew()))
skew = skew.stack().swaplevel(0,1)
skew.name = 'skew'

#年度最高价距离计算 1 - close/Max(high,252)
distance = data.groupby('asset').apply(lambda x:(1-x['close'] / x['high'].rolling(252).max()).droplevel('asset').resample('M').last())
distance =  distance.stack().swaplevel(0,1)
distance.name = 'distance'


#3个月动量
momentum = data.groupby('asset').apply(lambda x:x)


#行情数据降至月频率
bar_monthly = bar_resample(data[bars],frequency='M')





#数据合并对齐
merge_data = pd.concat([bar_monthly, skew, distance],axis=1)



## merged data (monthly)
处理和合并后行情和因子值的月度数据，

In [8]:
#合并数据示例
merge_data.dropna().head()

,,close,open,high,low,avg,volume,market_cap,skew,distance
date,asset,,,,,,,,,
2010-01-31,000001.XSHE,848.169983,958.390015,960.729980,805.169983,858.330017,24294088.0,634.532776,0.232038,0.188175
2010-02-28,000001.XSHE,877.479980,848.559998,884.130005,820.809998,875.919983,10662390.0,656.463684,0.886190,0.160121
2010-03-31,000001.XSHE,906.799988,877.479980,948.619995,866.539978,908.750000,15706730.0,678.394470,0.091379,0.132058
2010-04-30,000001.XSHE,803.609985,909.530029,932.590027,754.750000,798.919983,20433608.0,601.197876,-1.859854,0.230826
2010-05-31,000001.XSHE,684.400024,785.630005,793.450012,665.239990,694.559998,18574160.0,512.012390,-0.454606,0.344928


## forward return & factor quantile

In [9]:
mft = MultiFactorTesting(bar_data=merge_data.dropna()[bars],factor_data=merge_data.dropna()[['distance','skew']],add_shift=0)
clean_factor_return = mft.get_clean_factor_return(quantiles=3,labels={'distance':['Near','Med','Far'],'skew':['Low','Medium','High']})#clean_factor_return = mft.get_clean_factor_return(quantiles={'distance':3,'skew':3})

#因子分组和下期收益数据示例
clean_factor_return.head()

next_return distance    skew
date       asset                                    
2010-01-31 000001.XSHE     0.034081      Med    High
           000002.XSHE     0.013977      Far  Medium
           000004.XSHE     0.035993     Near  Medium
           000005.XSHE     0.035326      Far    High
           000006.XSHE     0.040107      Far    High

## market weights data

In [88]:
#利用流动市值计算market权重
w_mkt = merge_data.dropna()['market_cap'].groupby(level=0).apply(lambda x:x/x.sum())
w_mkt.head()

date        asset      
2010-01-31  000001.XSHE    0.004501
2010-02-28  000001.XSHE    0.004494
2010-03-31  000001.XSHE    0.004489
2010-04-30  000001.XSHE    0.004239
2010-05-31  000001.XSHE    0.003913
Name: market_cap, dtype: float32

In [89]:
## forward return of each cell(equal weight) 

In [90]:
#计算每个cell的平均下期收益(等权)
cell_returns_eq = clean_factor_return.groupby(level=0).apply(lambda x:x.groupby(['distance','skew']).mean()).unstack().unstack().droplevel(0,axis=1)
mkt_returns_eq = mft.compute_forward_bm_return()

# cell_returns_eq



In [91]:
## forward return of each cell(market_cap weight) 

In [92]:
#计算每个cell的平均下期收益(市值加权)
clean_factor_return_cap = pd.concat([clean_factor_return,mft.bar_data['market_cap']],axis=1)
cell_returns_cap = clean_factor_return_cap.groupby(level=0).apply(lambda x:x.groupby(['distance','skew']).apply(lambda x:(x['next_return']*x['market_cap']/x['market_cap'].sum()).sum()))
mkt_returns_cap = mft.compute_forward_bm_return(w_mkt=w_mkt)

# cell_returns_cap

In [93]:
#计算等权市场超额
excess_return_eq = cell_returns_eq.apply(lambda x:x-mkt_returns_eq)
#计算市值市场超额
excess_return_cap = cell_returns_cap.apply(lambda x:x-mkt_returns_cap)

## avg monthly excess return of all cells (equal weighted)

In [94]:
#等权市场平均超额收益(年化)
excess_return_eq.mean().unstack()

distance,Near,Med,Far
skew,,,
Low,0.002669,0.002188,0.004306
Medium,0.001639,-0.000536,0.002366
High,-0.002448,-0.006682,-0.003961


## avg monthly excess return of all cells (market Cap weighted)

In [95]:
#市值加权市场平均超额收益(年化)
excess_return_cap.mean().unstack().T

distance,Near,Med,Far
skew,,,
Low,0.000641,0.001269,0.001710
Medium,0.001253,-0.001209,0.000900
High,0.002168,-0.004415,-0.003854


## hit rate (equal weighted)

In [96]:
#等权市场hit rate
excess_return_eq.apply(lambda x:x[x>0].count() / x.count()).unstack()

distance,Near,Med,Far
skew,,,
Low,0.593548,0.606452,0.567742
Medium,0.522581,0.503226,0.509677
High,0.522581,0.258065,0.354839


## hit rate (market Cap weighted)

In [97]:
#市值加权市场hit rate
excess_return_cap.apply(lambda x:x[x>0].count() / x.count()).unstack().T

distance,Near,Med,Far
skew,,,
Low,0.561290,0.548387,0.477419
Medium,0.503226,0.496774,0.496774
High,0.535484,0.425806,0.419355


## avg number of stocks

In [98]:
# 不同cell的平均股票数量
clean_factor_return_copy = clean_factor_return.copy()
clean_factor_return_copy['next_return'] = 1
sort_count = clean_factor_return_copy.groupby(level=0).apply(lambda x:x.groupby(['distance','skew']).count()).unstack().unstack().droplevel(0,axis=1)
pprint(sort_count.mean().unstack())
pprint(f'股票总数平均值：{sort_count.mean().sum().sum()}')


distance        Near         Med         Far
skew                                        
Low       249.525641  343.717949  370.974359
Medium    329.807692  315.320513  311.211538
High      379.166667  298.814103  275.948718
'股票总数平均值：2874.4871794871797'


## avg proportion of all cells 

In [99]:
# 不同cell的平均股票数量占比
sort_count_ratio = sort_count.apply(lambda x:x/x.sum(),axis=1).mean().unstack()
sort_count_ratio

distance,Near,Med,Far
skew,,,
Low,0.085431,0.119772,0.130466
Medium,0.115157,0.109560,0.108038
High,0.132871,0.103884,0.094819


## performance of all cells (equal weighted)

In [100]:
from singletrader.performance.common import performance_indicator
round(performance_indicator((1+cell_returns_eq).cumprod(),freq=12),4).swaplevel(0,1,axis=1).sort_index(axis=1)

distance       Near                           Med                      \
skew            Low    Medium      High       Low    Medium      High   
累计收益         3.9449    3.3194    1.3177    3.4019    1.8823    0.1344   
年化收益         0.1317    0.1199    0.0672    0.1216    0.0854    0.0098   
年化波动率        0.2597    0.2486    0.2470    0.2727    0.2728    0.2661   
最大回撤         0.5083    0.5328    0.5950    0.4652    0.5062    0.6406   
夏普比(0.02)    0.4303    0.4020    0.1912    0.3725    0.2398   -0.0383   
年化收益/最大回撤    0.2591    0.2251    0.1130    0.2614    0.1687    0.0153   
累计交易周期     156.0000  156.0000  156.0000  156.0000  156.0000  156.0000   

distance        Far                      
skew            Low    Medium      High  
累计收益         4.6847    3.0973    0.5641  
年化收益         0.1440    0.1154    0.0352  
年化波动率        0.2976    0.3062    0.2996  
最大回撤         0.4241    0.4769    0.6006  
夏普比(0.02)    0.4167    0.3115    0.0509  
年化收益/最大回撤    0.3395    0.2419    0.0587  
累计交易周期     156.0000  156.0000  156.0000

## performance of all cells (market Cap weighted)

In [42]:
round(performance_indicator((1+cell_returns_cap).cumprod(),freq=12),4)#.swaplevel(0,1,axis=1)

distance       Near                           Med                      \
skew            Low    Medium      High       Low    Medium      High   
累计收益         0.7752    0.8719    1.2491    0.8474    0.2067   -0.2498   
年化收益         0.0454    0.0497    0.0648    0.0487    0.0147   -0.0220   
年化波动率        0.2050    0.2179    0.2090    0.2347    0.2366    0.2357   
最大回撤         0.4517    0.4719    0.4083    0.4380    0.4858    0.6101   
夏普比(0.02)    0.1241    0.1365    0.2141    0.1221   -0.0226   -0.1782   
年化收益/最大回撤    0.1006    0.1054    0.1586    0.1111    0.0302   -0.0361   
累计交易周期     156.0000  156.0000  156.0000  156.0000  156.0000  156.0000   

distance        Far                      
skew            Low    Medium      High  
累计收益         0.7722    0.5263   -0.2664  
年化收益         0.0453    0.0333   -0.0237  
年化波动率        0.2701    0.2787    0.2721  
最大回撤         0.4582    0.4787    0.6065  
夏普比(0.02)    0.0936    0.0476   -0.1606  
年化收益/最大回撤    0.0989    0.0695   -0.0391  
累计交易周期     156.0000  156.0000  156.0000

In [71]:
merge_data.to_csv(r'price_and_factor_data.csv')
clean_factor_return.to_csv(r'forward_return_and_quantile_data.csv')